# The Pollution-Routing Problem

We will consider the following (imaginary) scenario.

As part of the new organisation of the administrative regions in France, former regions Languedoc–Roussillon and Midi–Pyrénées merged to form the new *Région Occitanie*. 

The new region needs to deliver classified materials to all major cities (préfectures) of the new region, namely Albi, Auch, Cahors, Carcassonne, Foix, Mende, Montauban, Montpellier, Nîmes, Perpignan, Rodez, Tarbes, Toulouse. Each city will receive 200kg of classified materials.

The materials may depart from two top secret qualified facilities located in [Bugarach](https://www.theguardian.com/world/2012/nov/19/bugarach-french-village-survive-mayan-apocalypse) and [Espalion](https://fr.wikipedia.org/wiki/Les_Petits_Hommes).

Cost-reducing is a major concern in the new region, as well as environmental issues. The project manager for this mission needs to charter loading trucks departing from the three facilities and have them deliver in all the cities involved.

You have been hired to suggest an environment-friendly cost-effective solution.

The problem can be seen as an extension of the following academic problems:
- the [Bin Packing problem](https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_bin_packing);
- the [Vehicle Routing Problem](https://fr.wikipedia.org/wiki/Probl%C3%A8me_de_tourn%C3%A9es_de_v%C3%A9hicules);
- the [Travelling Salesman Problem](https://fr.wikipedia.org/wiki/Probl%C3%A8me_du_voyageur_de_commerce);
- and even more...

## Instructions

You are required to read the present file and get a good understanding of the basic problems of the introduction.  
You do have plenty of time, much more than necessary, to get prepared. However, **do no underestimate the work it represents**: be responsible, and make a good use of this time.

You may work in groups for this topic, ideally 3 students. You are free to form groups by yourself; however, in every group, there must be at most one *exchange student* and at least one student attending the *optimisation option* class in every group. You **must *share your workload*** as you are not supposed to duplicate analysis, code and results.

We will meet for a whole day on February 8th. During this day, we will share your views on the problem. At the end of the day, you will have chosen a particular aspect of the problem you would like to study more in depth and we will be able to discuss how to deal with this aspect of the problem.

You will have one more month to submit the final version of your project, **before February 21st, 9 a.m.**

For the evaluation, keep in mind that there is no such thing as *the result you are supposed to have*. The quality of your work will be evaluated from:
- how you state the problem you want to solve;
- how you simplify the problem in order to illustrate the problem you choose to solve;
- how you write your model in the optimisation libraries of your choice;
- how you write your conclusions and keep a critical eye on your work:  
  *"Our approach does not bring significant improvements over the naive implementation...",  
  "If we had access to this data, we could have..."*

You may submit your work in French or English.

## Contents

### Optimisation problems
*This part will be evaluated for 30% of the project.*
1. The Warehouse Location Problem
1. The Pollution-Routing Problem, minimising the distance
1. The Pollution-Routing Problem, minimising the weighted load
1. The Pollution-Routing Problem, minimising the pollution

### Tools
1. ipyleaflet
1. mapquest

Both tools are presented in a separate notebook. Read the [documentation](https://ipyleaflet.readthedocs.io/en/latest/) for a proper installation. (It should be fine on the school's computers)

### Case study
1. What is expected from you
1. Framework of **your** study;  
   *This part will be evaluated for 20% of the project.*
1. Your work;  
   *This part will be evaluated for 40% of the project.*

## Optimisation problems

### The Warehouse Location problem

In the Warehouse Location problem, a company considers opening warehouses at some candidate locations in order to supply its existing stores. Each possible warehouse has the same maintenance cost, and a capacity designating the maximum number of stores that it can supply. Each store must be supplied by exactly one open warehouse.

The supply cost to a store depends on the warehouse. The objective is to determine which warehouses to open, and which of these warehouses should supply the various stores, such that the sum of the maintenance and supply costs is minimized.

In [1]:
warehouses = [ "Paris", "Berlin", "Amsterdam", "Brussels", "London" ]
nbWarehouses = len(warehouses) # labeled from 0 to 4
nbStores = 10 # labeled from 0 to 9
capacity = [1, 4, 2, 2, 3] # capacity is indexed by warehouses

# maintenanceCost is fixed across warehouses
maintenanceCost = 30

# supplyCost is indexed by Stores(0..9) and the set of Warehouses
supplyCost = [ [ 20, 24, 11, 25, 30 ],
               [ 28, 27, 82, 83, 74 ],
               [ 74, 97, 71, 96, 70 ],
               [  2, 55, 73, 69, 61 ],
               [ 46, 96, 59, 83,  4 ],
               [ 42, 22, 29, 67, 59 ],
               [  1,  5, 73, 59, 56 ],
               [ 10, 73, 13, 43, 96 ],
               [ 93, 35, 63, 85, 46 ],
               [ 47, 65, 55, 71, 95 ] ]

You may address the problem by choosing a variable for each store:
```python
# stores[i] represents the id of the warehouse supplying store i
stores = [facile.variable(0, nbWarehouses - 1) for i in range(nbStores)]
```

The first constraint to be stated is that for each warehouse `i`, there is no more than `capacity[i]` variables taking value `i`. You may use the mechanism of *constraint reification* (transform a constraint into a variable) and build expressions such as:

```python
# the number of variables in stores that are equal to i
sum([p == i for p in stores])
```

As you build the objective function, you will need to sum the supplying costs for the appropriate store/warehouse pairs.  
For that purpose, you will need to index each line of `supplyCost` by a variable.  
Remember that you may wrap a Python array with `facile.array` so as to index the array by a variable:
```python
facile.array(supplyCost[i])[stores[i]]  # the cost to supply store i from warehouse stores[i]
```


----------------
**Note:** *You are required to print the solution in a human-readable format.*  

For example (but feel free to do better):
  
    Cost of the solution: 42
     - Stores of Paris: [0]
     - Stores of Berlin: [1, 2, 3, 4]
     - Stores of Amsterdam: [5]
     - Stores of Brussels: [6]
     - Stores of London: [7, 8, 9]

### The Pollution-Routing Problem

The Pollution-Routing Problem is defined on a complete graph $G = (\mathcal{N}, \mathcal{A})$ with $\mathcal{N} = \{0, 1, \cdots n\}$ and $\mathcal{A}$ the set of arcs defined between each pair of nodes. Node 0 is the depot.

There exist a homogeneous set of vehicles $\mathcal{K} = \{0, 1, \cdots m\}$, each with capacity $\mathcal{Q}$. The set $\mathcal{N}_0 = \mathcal{N} \backslash \{0\}$ is a customer set and every customer $i \in \mathcal{N}_0$ has demand $q_i$ and a request to be served within a prespecified time interval $[a_i, b_i]$. The time taken by a vehicle to serve customer $i$ is denoted by $t_i$ and the distance from $i$ to $j$ is denoted by $d_{ij}$.

Each vehicle emits a certain amount of GHG (Greenhouse Gas) when traveling over an arc $(i, j)$. This amount is dependent on a number of factors, such as load and speed, among others. Whereas some of these factors are fixed (e.g. gravity and slope), the load and speed variables can be controlled. The load of a vehicle is made up of empty load (*curb weight*) and carried load (*gross vehicle weight rating*).

The speed at which a vehicle travels on arc $(i, j)$ is constrained by a lower bound and an upper bound, denoted $l_{ij}$ and $u_{ij}$, respectively, usually imposed by traffic regulations.

Based on this introduction, we will consider three problems, all of which involve finding a set of tours for the set $\mathcal{K}$ of vehicles that start and end at the depot, such that the total load carried by any vehicles does not exceed its capacity Q. These problems differ with respect to their objective functions, some of which are related to the environment.

These objectives are:

- a distance-minimising objective function assuming constant speed;
- a weighted load-minimising objective function assuming constant speed;
- an energy-minimising objective function assuming speed as a decision.

A fourth problem is left as a possible problem to explore in the second part, and would consider the cost of emissions, drivers and fuel.

In [2]:
# We will work with PuLP for linear programming
import pulp
# A sane way to use arrays in Python
import numpy as np

We will use the following formulation of the problem. A binary variable $x_{ij}$ will be equal to 1 if a vehicle travels on arc $(i, j) \in \mathcal{A}$.  
For such a given arc, $f_{ij}$ will represent the amount of commodity flowing and $v_{ij}$ the speed at which a vehicle travels on this arc.

#### Minimising the distance

The first problem only considers a distance minimisation, that is a minimisation of the following expression

$$\sum_{i,j \in \mathcal{A}} d_{ij}\,x_{ij}$$

where $d_{ij}$ represents the distance of arc $(i, j)$, subject to several constraints:

- **(C1)**: only $m$ trucks leave from the depot (we will start with $m=1$);
- **(C2)**: there is only one "way" to arrive to each client;
- **(C3)**: there is only one "way" to depart from each client;
- **(C4)**: an amount of merchandise $q_i$ is left at client $i$ (so the truck weights less after leaving $i$);
- **(C5)**: a truck cannot carry more than its capacity.

In [3]:
n = 4  # number of nodes
m = 1  # number of trucks
w = 3  # curb weight


# We consider a fully connected graph of four nodes placed in a rectangle
# Node 0 is the depot
# Distances are in km
sq_5 = 100 * np.sqrt(5)
distance = np.array([[0, 200, sq_5, 100],
                     [200, 0, 100, sq_5],
                     [sq_5, 100, 0, 200],
                     [100, sq_5, 200, 0]])

# Everybody gets 1 ton
quantity = np.array([0, 1, 1, 1])
Q = np.sum(quantity)



<div class="alert alert-warning">
**Problem:** Implement the problem and print a sequence of nodes that minimises the total distance.
</div>

For your convenience, the following variable arrays have already been created.

In [4]:
prob = pulp.LpProblem("The Pollution-Routing Problem (distance)",pulp.LpMinimize)

/home/lena/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
def var_array(pattern):
    """Creates a two-dimension array of variables created by function pattern(i, j).
    
    The value is automatically set to integer 0 if i == j. 
    The array is transformed into an np.array.
    """
    array = [
        [
            pattern(i, j) if i != j else 0
            for j in range(n)
        ] for i in range(n)
    ]
    return np.array(array)

# We create the x_ij variables as binary variables (0 if no way exists)
x = var_array(lambda i, j: pulp.LpVariable("x_%d_%d" % (i, j), cat = pulp.LpBinary))
# f_ij represents the weight transiting from i to j (0 if not transiting)
f = var_array(lambda i, j: pulp.LpVariable("f_%d_%d" % (i, j), 0, Q))

In [6]:
# Constrait 1 : only 𝑚 trucks leave from the depot (we will start with 𝑚=1)
prob += sum(x[0,j] for j in range(1,n)) == m

# Constraint 2 : there is only one "way" to arrive to each client
for i in range(1,n):
    prob += sum(x[j,i] for j in range(n)) == 1


#Constraint 3 : there is only one "way" to depart from each client
for i in range(1,n):
    prob += sum(x[i,j] for j in range(n)) == 1
    
    
#Constraint 4: an amount of merchandise 𝑞𝑖 is left at client 𝑖 (so the truck weights less after leaving 𝑖);
for i in range(1,n):
    prob += sum(f[j,i] for j in range(n)) == sum(f[i,j] for j in range(n)) + quantity[i]

# Constraint 5 : a truck cannot carry more than its capacity.
for i in range(1,n):
    for j in range(n):
        prob += (f[i,j] <= int(Q))
    
for i in range(n):
    for j in range(n):
        prob += int(Q)*x[i,j] >= f[i][j]


In [7]:
#Definition de la fonction objective distance
prob += pulp.lpSum(distance[i,j]*x[i,j] for i in range(n) for j in range(n))

In [8]:
# Résolution du problème de contraintes
status_distance = prob.solve()
print(pulp.LpStatus[status_distance])

Optimal


In [9]:
print(pulp.value(prob.objective))

for v in prob.variables():
    print(v.name, "=", v.varValue)

600.0
f_0_1 = 3.0
f_0_2 = 0.0
f_0_3 = 0.0
f_1_0 = 0.0
f_1_2 = 2.0
f_1_3 = 0.0
f_2_0 = 0.0
f_2_1 = 0.0
f_2_3 = 1.0
f_3_0 = 0.0
f_3_1 = 0.0
f_3_2 = 0.0
x_0_1 = 1.0
x_0_2 = 0.0
x_0_3 = 0.0
x_1_0 = 0.0
x_1_2 = 1.0
x_1_3 = 0.0
x_2_0 = 0.0
x_2_1 = 0.0
x_2_3 = 1.0
x_3_0 = 1.0
x_3_1 = 0.0
x_3_2 = 0.0


------------------------

In [10]:
# Pretty print of the sequence

def where(x, n, i):
    """Builds a matrix to detect where a non-null value exists."""
    return np.array([(x[i][j].value() > 0) 
                     if isinstance(x[i][j], pulp.pulp.LpVariable)
                      else False for j in range(n)])

matrix = [np.where(where(x, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]

print ("Sequence:\t", sequence)
def value(x, n, i):
    """Builds a matrix to print values where they exist."""
    return np.array([(x[i][j].value()
                      if isinstance(x[i][j], pulp.pulp.LpVariable)
                      else None) for j in range(n)])

f_r = np.array([value(f, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)

Sequence:	 [0, 1, 2, 3, 0]
Commodity flow:	 [3. 2. 1. 0.]


#### Minimising the weighted load

The second model is based on the first one, with a different objective function:

$$\sum_{i,j \in \mathcal{A}} d_{ij}\, (w\, x_{ij} + f_{ij})$$

where $w = 3\,t$ is the curb weight.

<div class="alert alert-warning">
**Problem:** Implement the problem and print a sequence of nodes that minimises the weighted load.
</div>


In [11]:
prob_weight = pulp.LpProblem("The Pollution-Routing Problem (weight)",pulp.LpMinimize)

In [12]:
def var_array(pattern):
    """Creates a two-dimension array of variables created by function pattern(i, j).
    
    The value is automatically set to integer 0 if i == j. 
    The array is transformed into an np.array.
    """
    array = [
        [
            pattern(i, j) if i != j else 0
            for j in range(n)
        ] for i in range(n)
    ]
    return np.array(array)

# We create the x_ij variables as binary variables (0 if no way exists)
xw = var_array(lambda i, j: pulp.LpVariable("x_%d_%d" % (i, j), cat = pulp.LpBinary))
# f_ij represents the weight transiting from i to j (0 if not transiting)
fw = var_array(lambda i, j: pulp.LpVariable("f_%d_%d" % (i, j), 0, Q))

In [13]:
# Constrait 1 : only 𝑚 trucks leave from the depot (we will start with 𝑚=1)
prob_weight += sum(xw[0,j] for j in range(1,n)) == m

# Constraint 2 : there is only one "way" to arrive to each client
for i in range(1,n):
    prob_weight  += sum(xw[j,i] for j in range(n)) == 1


#Constraint 3 : there is only one "way" to depart from each client
for i in range(1,n):
    prob_weight  += sum(xw[i,j] for j in range(n)) == 1
    
    
#Constraint 4: an amount of merchandise 𝑞𝑖 is left at client 𝑖 (so the truck weights less after leaving 𝑖);
for i in range(1,n):
    prob_weight  += sum(fw[j,i] for j in range(n)) == sum(fw[i,j] for j in range(n)) + quantity[i]

# Constraint 5 : a truck cannot carry more than its capacity.
for i in range(1,n):
    for j in range(n):
        prob_weight  += (fw[i,j] <= int(Q))
    
for i in range(n):
    for j in range(n):
        prob_weight  += int(Q)*xw[i,j] >= fw[i][j]


In [14]:
# Fonction objective poids
prob_weight += pulp.lpSum(distance[i,j]*(w*xw[i,j]+fw[i,j]) for i in range (n) for j in range(n))

In [15]:
status_weight = prob_weight.solve()
print(pulp.LpStatus[status_weight])


Optimal


----------

In [16]:
matrix = [np.where(where(xw, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]
    
print ("Sequence:\t", sequence)


f_r = np.array([value(fw, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)

print ("Objective function:\t", pulp.value(prob_weight.objective))

Sequence:	 [0, 3, 2, 1, 0]
Commodity flow:	 [3. 2. 1. 0.]
Objective function:	 2600.0


<div class="alert alert-warning">
**Problem:** Compare the results of the two preceding models for the following demand.
</div>

In [17]:
quantity = np.array([0, .25, 3.5, .25])
Q = np.sum(quantity)

In [18]:
prob2_distance = pulp.LpProblem("The Pollution-Routing Problem (distance)",pulp.LpMinimize)

In [19]:
prob2_weight = pulp.LpProblem("The Pollution-Routing Problem (weight)",pulp.LpMinimize)

In [20]:
def var_array(pattern):
    """Creates a two-dimension array of variables created by function pattern(i, j).
    
    The value is automatically set to integer 0 if i == j. 
    The array is transformed into an np.array.
    """
    array = [
        [
            pattern(i, j) if i != j else 0
            for j in range(n)
        ] for i in range(n)
    ]
    return np.array(array)

# We create the x_ij variables as binary variables (0 if no way exists)
xd = var_array(lambda i, j: pulp.LpVariable("x_%d_%d" % (i, j), cat = pulp.LpBinary))
# f_ij represents the weight transiting from i to j (0 if not transiting)
fd = var_array(lambda i, j: pulp.LpVariable("f_%d_%d" % (i, j), 0, Q))

In [21]:
def var_array(pattern):
    """Creates a two-dimension array of variables created by function pattern(i, j).
    
    The value is automatically set to integer 0 if i == j. 
    The array is transformed into an np.array.
    """
    array = [
        [
            pattern(i, j) if i != j else 0
            for j in range(n)
        ] for i in range(n)
    ]
    return np.array(array)

# We create the x_ij variables as binary variables (0 if no way exists)
xw2 = var_array(lambda i, j: pulp.LpVariable("x_%d_%d" % (i, j), cat = pulp.LpBinary))
# f_ij represents the weight transiting from i to j (0 if not transiting)
fw2 = var_array(lambda i, j: pulp.LpVariable("f_%d_%d" % (i, j), 0, Q))

In [22]:
# Constrait 1 : only 𝑚 trucks leave from the depot (we will start with 𝑚=1)
prob2_weight += sum(xw2[0,j] for j in range(1,n)) == m

# Constraint 2 : there is only one "way" to arrive to each client
for i in range(1,n):
    prob2_weight  += sum(xw2[j,i] for j in range(n)) == 1


#Constraint 3 : there is only one "way" to depart from each client
for i in range(1,n):
    prob2_weight  += sum(xw2[i,j] for j in range(n)) == 1
    
    
#Constraint 4: an amount of merchandise 𝑞𝑖 is left at client 𝑖 (so the truck weights less after leaving 𝑖);
for i in range(1,n):
    prob2_weight  += sum(fw2[j,i] for j in range(n)) == sum(fw2[i,j] for j in range(n)) + quantity[i]

# Constraint 5 : a truck cannot carry more than its capacity.
for i in range(1,n):
    for j in range(n):
        prob2_weight  += (fw2[i,j] <= int(Q))
    
for i in range(n):
    for j in range(n):
        prob2_weight  += int(Q)*xw2[i,j] >= fw2[i][j]


In [23]:
# Constrait 1 : only 𝑚 trucks leave from the depot (we will start with 𝑚=1)
prob2_distance += sum(xd[0,j] for j in range(1,n)) == m

# Constraint 2 : there is only one "way" to arrive to each client
for i in range(1,n):
    prob2_distance  += sum(xd[j,i] for j in range(n)) == 1


#Constraint 3 : there is only one "way" to depart from each client
for i in range(1,n):
    prob2_distance  += sum(xd[i,j] for j in range(n)) == 1
    
    
#Constraint 4: an amount of merchandise 𝑞𝑖 is left at client 𝑖 (so the truck weights less after leaving 𝑖);
for i in range(1,n):
    prob2_distance  += sum(fd[j,i] for j in range(n)) == sum(fd[i,j] for j in range(n)) + quantity[i]

# Constraint 5 : a truck cannot carry more than its capacity.
for i in range(1,n):
    for j in range(n):
        prob2_distance  += (fd[i,j] <= int(Q))
    
for i in range(n):
    for j in range(n):
        prob2_distance  += int(Q)*xd[i,j] >= fd[i][j]


In [24]:
#Definition de la fonction objective distance
prob2_distance += pulp.lpSum(distance[i,j]*xd[i,j] for i in range(n) for j in range(n))
# Fonction objective poids
prob2_weight += pulp.lpSum(distance[i,j]*(w*xw2[i,j]+fw2[i,j]) for i in range (n) for j in range(n))

In [25]:
# Résolution du problème de contraintes
status2_distance = prob2_distance.solve()
print("The problem distance is {}".format(pulp.LpStatus[status2_distance]))

status2_weight = prob2_weight.solve()
print("The problem weight is {}".format(pulp.LpStatus[status2_weight]))

The problem distance is Optimal
The problem weight is Optimal


In [26]:
print('-------------------------------')
print('For the distance')
print('')
print('With int quantity')
matrix = [np.where(where(x, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]

print ("Sequence:\t", sequence)
f_r = np.array([value(f, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)

print('')
print('With float quantity')
matrix = [np.where(where(xd, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]

print ("Sequence:\t", sequence)
f_r = np.array([value(fd, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)


print('-------------------------------')
print('For the weight')
print('')
print('With int quantity')
matrix = [np.where(where(xw, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]

print ("Sequence:\t", sequence)
f_r = np.array([value(fw, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)








print('')
print('With float quantity')
matrix = [np.where(where(xw2, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]

print ("Sequence:\t", sequence)
f_r = np.array([value(fw2, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)


-------------------------------
For the distance

With int quantity
Sequence:	 [0, 1, 2, 3, 0]
Commodity flow:	 [3. 2. 1. 0.]

With float quantity
Sequence:	 [0, 3, 2, 1, 0]
Commodity flow:	 [4.   3.75 0.25 0.  ]
-------------------------------
For the weight

With int quantity
Sequence:	 [0, 3, 2, 1, 0]
Commodity flow:	 [3. 2. 1. 0.]

With float quantity
Sequence:	 [0, 2, 1, 3, 0]
Commodity flow:	 [4.   0.5  0.25 0.  ]


---------------

#### Minimising the energy

The third model takes into account the total energy spent on a solution.

We approximate the total energy spent on arc $(i,j)$ as $ \alpha_{ij}\, (w + f_{ij})\,d_{ij} + \beta \, v_{ij}^2 \, d_{ij}$ where:
- $\alpha_{ij} = a + g\,\sin \theta_{ij} + g\, C_r\,\cos\theta_{ij}$ is an *arc specific constant*:  
  $a$ measures the acceleration we consider equal to zero (constant speed), $\theta_{ij}$ the average road angle on arc $(i,j)$ and $C_r$ the rolling resistance.
- $\beta = \dfrac{1}{2}\,C_d\,A\,\rho$ is a *vehicle specific constant*:  
  $C_d$ measures the drag resistance, $A$ the frontal surface area of the vehicle and $\rho$ the air density.

As $v_{ij}^2$ is not an acceptable option for linear programming, we need to linearise the variables. We introduce a set of speed levels $\mathcal{R} = \{1, 2, \cdots \}$ between $l_{ij} = l$ and $u_{ij} = u$ and introduce binary variables $z_{ij}^r$ being true if a vehicle travels at speed level $r\in\mathcal{R}$ on arc $(i,j)$.

So the new objective function is:

$$\sum_{i,j \in \mathcal{A}} d_{ij}\, \left(\alpha_{ij}\, \left(w\, x_{ij} + f_{ij}\right) + \beta \left(\sum_{r\in\mathcal{R}} (v^r)^2 z_{ij}^r\right)\right)$$

Note that a new constraint **(C6)** must ensure that:

$$\sum_{r\in\mathcal{R}} z_{ij}^r = x_{ij}$$

In [31]:
a  = 0     # acceleration
g  = 9.81  # gravity
cr = 0.01  # rolling resistance
cd = 0.7   # drag resistance
A  = 5     # frontal surface resistance, m²
rho = 1.2041 # air density, kg/m³

## Speed limitations
R = 10     # Discretisation of speed
l = 40     # km/h
u = 70     # km/h


<div class="alert alert-warning">
**Problem:** Implement the problem and print a sequence of nodes that minimises the energy along with the corresponding average speeds.
</div>

In [32]:
pb = pulp.LpProblem("energy", pulp.LpMinimize)


theta = np.zeros((n,n))
alpha = a + g * np.sin(theta) + g * cr * np.cos(theta)
beta = 0.5 * cd * A * rho

# We create the x_ij variables as binary variables (0 if no way exists)
x = var_array(lambda i, j: pulp.LpVariable("x_%d_%d" % (i, j), cat = pulp.LpBinary))
# f_ij represents the weight transiting from i to j (0 if not transiting)
f = var_array(lambda i, j: pulp.LpVariable("f_%d_%d" % (i, j), 0, Q))

# Discretization of speed
z_lambda = lambda i, j, r: pulp.LpVariable("z_%d_%d_%d" % (i, j, r), cat = pulp.LpBinary)

z = [[[z_lambda(i, j, r) if i != j else 0
       for r in range(R)]  # discretisation of speed
      for j in range(n)]
     for i in range(n)]
z = np.array(z)

# Computation of v_2 = sum(v_2 * z_ij)
v = l + np.arange(R) * (u - l)/R
v += (v[1] - v[0])/2
v_2 = np.sum(v ** 2 * z, axis=2)

----------


In [33]:
# Constrait 1 : only 𝑚 trucks leave from the depot (we will start with 𝑚=1)
pb += sum(x[0,j] for j in range(1,n)) == m

# Constraint 2 : there is only one "way" to arrive to each client
for i in range(1,n):
    pb  += sum(x[j,i] for j in range(n)) == 1


#Constraint 3 : there is only one "way" to depart from each client
for i in range(1,n):
    pb  += sum(x[i,j] for j in range(n)) == 1
    
    
#Constraint 4: an amount of merchandise 𝑞𝑖 is left at client 𝑖 (so the truck weights less after leaving 𝑖);
for i in range(1,n):
    pb  += sum(f[j,i] for j in range(n)) == sum(f[i,j] for j in range(n)) + quantity[i]

# Constraint 5 : a truck cannot carry more than its capacity.
for i in range(1,n):
    for j in range(n):
        pb  += (f[i,j] <= int(Q))
    
for i in range(n):
    for j in range(n):
        pb  += int(Q)*x[i,j] >= f[i][j]
        
# Constraint 6 : fix the velocity on every edges.
for i in range(n):
    for j in range (n):
        pb += sum(z[i,j,r] for r in range(R)) == x[i,j]

In [ ]:
#Definition of the objective function


--------------------

In [30]:
matrix = [np.where(where(x, n, i))[0][0] for i in range(n)]
sequence = [0]
nxt = matrix[0]

for p in range(n):
    sequence.append(nxt)
    nxt = matrix[nxt]
    
print ("Sequence:\t", sequence)

def value(x, n, i):
    """Builds a matrix to print values where they exist."""
    return np.array([(x[i][j].value()
                      if isinstance(x[i][j], pulp.pulp.LpVariable)
                      else None) for j in range(n)])

f_r = np.array([value(f, n, i)[matrix[i]] for i in range(n)])
f_r = f_r[sequence[:-1]]

print ("Commodity flow:\t", f_r)

z_r = [[[z[i][j][r].value()  if isinstance(z[i][j][r],pulp.pulp.LpVariable) else 0
         for r in range(R)] for j in range(n)] for i in range(n)]

speeds = []
for i in range(n):
    pos = np.array([sum(p) for p in z_r[i]])
    idx = np.array(z_r[i][np.where(pos>0)[0][0]])
    speeds.append(v[np.where(idx>0)[0]][0])

print("Average speed:\t", speeds)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

## Case studies

### What you are expected to work on

The exercices you submitted should help you design a basic solution of the problem.  
Still, there are many additional parameters that you may take into account.

You will have to choose one or several additional aspects and take them into account for your case study.

You may pick in the following list, which is **not** limitative. Feel free to address your own interrogations.

- **Take service time into account.**  
  Some clients have different constraints on their opening times.  
  Also, it takes time to empty the truck at destination.  
  Design a model to take this parameter into account.  
  Analyse the impact on speed on each segment.

- **Diesel vs. regular.**  
  Diesel is cheaper, but the gas emissions may be more taxed.
  
- **Faster vs. shorter itinerary.**  
  Google API lets you choose between different options when you compute itineraries.

- **Cost of labour.**  
  Drivers cost as they are on the road. Also there is a legal time for rest that drivers must abide by.  
  
- **Traffic.**  
  Google API lets you compute itineraries at different times of the day.

- **Altitude.**  
  It may be cheaper to access cities in altitude when the truck is not fully loaded.
  
- **Number of vehicles.**  
  The first model may involve one truck per secret facility. Is it relevant to dispatch more trucks?

- **Impact of an unforeseen accident on a major axis.**  
  Imagine that A61 road is closed for trucks (or any other scenario).  
  What impact does it have on your solution?

### Framework of your study

<div class="alert alert-warning" style="margin-top: 1em">
*This part will be evaluated for 30% of the project.*
</div>

- Define the aspects you want to take into account for your project.
- Specify a case study, reasonable specifications, and additional data you may need (and find!) and exploit.
- Explain the kind of results you may reasonably expect.
